In [1]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModel
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, metrics, regularizers
import pandas as pd
import numpy as np
import tensorflow as tf
import sys

/home/yu/kaggle-tweet-sentiment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-29 18:52:27.096058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738194747.107431  133331 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738194747.112307  133331 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 18:52:27.124367: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

In [2]:
train = pd.read_csv("data1/train.csv").dropna()
train, test = train_test_split(train, test_size=0.15)
print(train.info())
print(test.info())
# TODO see if selected_text needs to be fixed to word boundaries

<class 'pandas.core.frame.DataFrame'>
Index: 23358 entries, 15000 to 16948
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         23358 non-null  object
 1   text           23358 non-null  object
 2   selected_text  23358 non-null  object
 3   sentiment      23358 non-null  object
dtypes: object(4)
memory usage: 912.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 4122 entries, 658 to 25970
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         4122 non-null   object
 1   text           4122 non-null   object
 2   selected_text  4122 non-null   object
 3   sentiment      4122 non-null   object
dtypes: object(4)
memory usage: 161.0+ KB
None


In [3]:
max_text_len = 128
span_indices_size = max_text_len * (max_text_len + 1) // 2
span_indices = np.zeros((max_text_len, max_text_len), dtype=int)
span_indices[np.triu_indices(max_text_len, m=max_text_len)] = np.arange(span_indices_size)
span_indices = np.flip(span_indices, axis=1)
inference_batch_size = 2000

def tokenize(texts, padding=True):
    padding = "max_length" if padding else "longest"
    return tokenizer(texts, padding=padding, max_length=max_text_len, return_tensors="tf")

def detokenize(ids, skip_special=True):
    return tokenizer.batch_decode(ids, skip_special_tokens=skip_special)

# https://stackoverflow.com/a/7100681
def rolling_window(a, size):
    shape = a.shape[:-1] + (a.shape[-1] - size + 1, size)
    strides = a.strides + (a. strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def find_span(texts, spans):
    texts = np.array(texts)
    spans = np.array(spans)[:, 1:]
    sizes = spans.argmin(axis=1) - 1
    span_class = np.zeros((texts.shape[0], 1))
    for i, text in enumerate(texts):
        window_size = sizes[i]
        matched_window = rolling_window(texts[i], window_size) == spans[i][:window_size]
        window_from = matched_window.all(axis=1).argmax()
        window_to = window_size - 1
        try:
            span_class[i] = span_indices[window_from, window_to]
        except IndexError as e:
            print(window_from, window_to, span_indices[window_from, window_to])
            raise e
    return span_class

def span_range(index):
    [d1], [d2] = np.where(span_indices == index)
    return d1, d1 + d2 + 1


def eprint(*args, **kwargs):
    print(*args, **kwargs, file=sys.stderr)

def print_mem_info():
    info = tf.config.experimental.get_memory_info("/gpu:0")
    eprint(f"GPU memory usage {(info["current"] / info["peak"]):.2f}.")

def context_embeddings(texts, sentiments, selected_texts=None):
    text_tokens = tokenize(texts)
    text_embeddings = encoder(**text_tokens).last_hidden_state
    sentiment_tokens = tokenize(sentiments)
    sentiment_embeddings = encoder(**sentiment_tokens).last_hidden_state[:, 1:2, :]  # only need embedding for one word
    embeddings = tf.concat((text_embeddings, sentiment_embeddings), 1)
    if selected_texts is None:
        return embeddings
    else:
        selected_tokens = tokenize(selected_texts)
        targets = find_span(text_tokens["input_ids"], selected_tokens["input_ids"])
        return embeddings, targets

def dataset_generator(texts, sentiments, selected_texts=None):
    def gen():
        total = len(texts)
        yielded = 0
        while yielded < total:
            batch_i = yielded % inference_batch_size
            # batch inferencing
            if batch_i == 0:
                end = min(total, yielded + inference_batch_size)
                if selected_texts is None:
                    embeddings = context_embeddings(texts[yielded:end], sentiments[yielded:end])
                else:
                    embeddings, targets = context_embeddings(texts[yielded:end], sentiments[yielded:end], selected_texts[yielded:end])
            # feed the generator
            if selected_texts is None:
                yield embeddings[batch_i]
            else:
                yield embeddings[batch_i], targets[batch_i]
            yielded += 1
    return gen

Naive implementation, flattened embeddings into FFN with one hidden layer.

In [4]:
inputs = keras.Input(shape=(1 + max_text_len, 768))
softmax = layers.Dense(span_indices_size, activation="softmax", kernel_regularizer="l1l2", bias_regularizer="l2")(
          layers.Dense(128, activation="relu", kernel_regularizer="l1l2", bias_regularizer="l2")(
          layers.Flatten()(
          inputs)))
ffn = keras.Model(inputs=inputs, outputs=softmax)
ffn.compile(
    optimizer=optimizers.Adam(),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
encoder = TFAutoModel.from_pretrained("distilbert-base-uncased")
dataset_train = tf.data.Dataset.from_generator(
    dataset_generator(train.text.to_list(), train.sentiment.to_list(), train.selected_text.to_list()),
    output_signature=(tf.TensorSpec(shape=(1 + max_text_len, 768)), tf.TensorSpec(shape=(1,)))
).shuffle(inference_batch_size)
dataset_test = tf.data.Dataset.from_generator(
    dataset_generator(test.text.to_list(), test.sentiment.to_list(), test.selected_text.to_list()),
    output_signature=(tf.TensorSpec(shape=(1 + max_text_len, 768)), tf.TensorSpec(shape=(1,)))
)
history = ffn.fit(dataset_train.batch(64), epochs=10, validation_data=dataset_test.batch(64))

I0000 00:00:1738194748.896815  133331 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21759 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.

Epoch 1/10


I0000 00:00:1738194758.380484  133521 service.cc:148] XLA service 0x7f8a780047c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738194758.380500  133521 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-01-29 18:52:38.396896: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1738194758.453961  133521 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-29 18:52:39.076439: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_297_0', 52 bytes spill stores, 52 bytes spill loads

2025-01-29 18:52:39.252803: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_297', 80 bytes

      5/Unknown 10s 36ms/step - loss: 8.5449 - sparse_categorical_accuracy: 0.0149   

I0000 00:00:1738194760.539315  133521 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

2025-01-29 19:01:29.223146: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17488572742734630363
2025-01-29 19:01:29.223179: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4530719411164311286


360/365 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.5402 - sparse_categorical_accuracy: 0.8454 

2025-01-29 19:02:19.245489: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17488572742734630363
2025-01-29 19:02:19.245519: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4530719411164311286


365/365 ━━━━━━━━━━━━━━━━━━━━ 59s 141ms/step - loss: 0.5403 - sparse_categorical_accuracy: 0.8454 - val_loss: 5.4961 - val_sparse_categorical_accuracy: 0.3993


2025-01-29 19:02:27.937530: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17488572742734630363
2025-01-29 19:02:27.937561: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4530719411164311286


In [36]:
# https://www.kaggle.com/competitions/tweet-sentiment-extraction
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

score = 0
total = test.shape[0]
y = ffn.predict(dataset_test.batch(64))
y = tf.math.argmax(y, axis=1)
for i, y in enumerate(y):
    span_from, span_to = span_range(y)
    y_str = test.text.iloc[i][span_from:span_to]
    t_str = test.selected_text.iloc[i]
    score += 1 / total * jaccard(y_str, t_str)
score

65/65 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step


0.10057695953347279